In [1]:
import wandb

wandb.login()

wandb: Currently logged in as: claudios (use `wandb login --relogin` to force relogin)


True

In [2]:
# -*- coding: utf-8 -*-
"""Tokenizer-Quicktour

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1sVUP6YwJhocDn6SS_2Na2KIKU5iBugDi
"""

#!mkdir data

#!pip install transformers tokenizers datasets

import torch
print(f"CUDA: {torch.cuda.is_available()}")
#torch.cuda.set_device(5)

CUDA: True


In [ ]:
#!nvidia-smi

In [71]:
import numpy as np

In [14]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers import normalizers
from tokenizers.normalizers import Lowercase, NFD, StripAccents
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.processors import BertProcessing
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Split
from tokenizers.normalizers import Strip
from tokenizers import Regex

In [27]:
exp = Regex("(^((\w)+(?=\s)))|((\[ENTRY\]\ (\w|\||\.)+)\s)|((\[CALL\]\ (\w|\||\.|\s)+)(?=\ \[))+|(\[EXIT\])")

pre_tokenizer = Split(pattern=exp, behavior="removed",invert=True)
#print(pre_tokenizer.pre_tokenize_str("performExpensiveLogSetup [ENTRY] void [CALL] java.io.PrintStream println java.lang.String void [CALL] java.lang.Math pow double|double double [CALL] java.lang.Math sqrt double double [CALL] java.io.PrintStream println java.lang.String void [EXIT]"))

trace_tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
trace_tokenizer.add_special_tokens(["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
trace_tokenizer.normalizer = Strip()
trace_tokenizer.pre_tokenizer = pre_tokenizer
# trace_tokenizer.post_processor = BertProcessing(sep=("[SEP]", 0),cls=("[CLS]", 1))
VOCAB_SIZE = 9587
#trace_tokenizer.add_tokens([' '])
trainer = WordLevelTrainer(
    vocab_size=VOCAB_SIZE, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)
files = ["work/projsplit/google.txt"]
trace_tokenizer.train(files, trainer)

#trace_tokenizer.save("trace_tokenizer_google_big.json")

# print(pre_tokenizer.pre_tokenize_str("performExpensiveLogSetup [ENTRY] void [CALL] java.io.PrintStream println java.lang.String void [CALL] java.lang.Math pow double|double double [CALL] java.lang.Math sqrt double double [CALL] java.io.PrintStream println java.lang.String void [EXIT]"))

In [65]:
encoding = trace_tokenizer.encode("[EXIT]")
encoding.tokens


['[CLS]', '[EXIT]', '[SEP]']

In [62]:
trace_tokenizer.post_processor = BertProcessing(sep=("[SEP]", 0),cls=("[CLS]", 1))

In [60]:
bproc = BertProcessing(sep=("[SEP]", 0),cls=("[CLS]", 1)).process(encoding)

In [61]:
bproc.tokens

['[CLS]', '[EXIT]', '[SEP]']

In [71]:
from transformers import BertConfig, BertForMaskedLM
scale_factor = 1 
config = BertConfig(
    vocab_size=9587, #VOCAB_SIZE
    max_position_embeddings=int(768*scale_factor),
    intermediate_size=int(2048*scale_factor),
    hidden_size=int(512*scale_factor),
    num_attention_heads=8,
    num_hidden_layers=6,
    type_vocab_size=5,
    hidden_dropout_prob=0.1,
    attention_probs_dropout_prob=0.1,
)

from transformers import PreTrainedTokenizerFast
#tokenizer_object=trace_tokenizer
fast_tokenizer = PreTrainedTokenizerFast(tokenizer_file="trace_tokenizer_google.json",
                                         return_special_tokens_mask=True, return_token_type_ids=True)

fast_tokenizer.add_special_tokens({'pad_token': '[PAD]', 'mask_token': '[MASK]', 'sep_token': '[SEP]'})

from datasets import load_dataset

In [72]:
fast_tokenizer.sep_token

'[SEP]'

In [73]:
fast_tokenizer.decode(fast_tokenizer.encode("[EXIT]"))

'[CLS] [EXIT] [UNK]'

In [13]:
fast_tokenizer.vocab_size

9582

In [29]:
small_train_dataset = load_dataset('text', data_files='work/projsplit/google.txt', split='train[:5%]')

Using custom data configuration default-fd3e663e823696b8
Reusing dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-fd3e663e823696b8/0.0.0/d86c40dad297bdddf277b406c6a59f0250b5318c400bf23d420a31aff88c84c4)


In [30]:
small_train_dataset

Dataset({
    features: ['text'],
    num_rows: 3376066
})

In [52]:
smaller_test_dataset = load_dataset('text', data_files='work/projsplit/google.txt', split='train[:1500]')

Using custom data configuration default-fd3e663e823696b8
Reusing dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-fd3e663e823696b8/0.0.0/d86c40dad297bdddf277b406c6a59f0250b5318c400bf23d420a31aff88c84c4)


In [51]:
smaller_test_dataset

Dataset({
    features: ['text'],
    num_rows: 67519820
})

In [59]:
# small_eval_dataset

model = BertForMaskedLM(config)
model.tokenizer = fast_tokenizer

def preprocess_function(examples):
    return fast_tokenizer(examples["text"], max_length = 512, truncation=True, padding=True)

In [60]:
encoded_dataset_train = small_train_dataset.map(preprocess_function, batched=True)

  0%|          | 0/3377 [00:00<?, ?ba/s]

In [11]:
encoded_dataset_train

Dataset({
    features: ['attention_mask', 'input_ids', 'text', 'token_type_ids'],
    num_rows: 3376066
})

In [61]:
encoded_dataset_test = smaller_test_dataset.map(preprocess_function, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

In [62]:
encoded_dataset_test = encoded_dataset_test.map(lambda x: {'func_name': x["text"].split(' ')[0]})

  0%|          | 0/1500 [00:00<?, ?ex/s]

In [63]:
encoded_dataset_train = encoded_dataset_train.map(lambda x: {'func_name': x["text"].split(' ')[0]})

  0%|          | 0/3376066 [00:00<?, ?ex/s]

In [ ]:
# trace_tokenizer.encode("filterSuite [ENTRY] junit.framework.TestSuite junit.framework.TestSuite [CALL] java.util.Enumeration hasMoreElements  boolean [CALL] java.util.Enumeration nextElement  java.lang.Object [CALL] java.util.Enumeration hasMoreElements  boolean [CALL] java.util.Enumeration nextElement  java.lang.Object [CALL] java.util.Enumeration hasMoreElements  boolean [CALL] java.util.Enumeration nextElement  java.lang.Object [CALL] java.util.Enumeration hasMoreElements  boolean [CALL] java.util.Enumeration nextElement  java.lang.Object [CALL] java.util.Enumeration hasMoreElements  boolean [CALL] java.util.Enumeration nextElement  java.lang.Object [CALL] java.util.Enumeration hasMoreElements  boolean [CALL] java.util.Enumeration nextElement  java.lang.Object [CALL] java.util.Enumeration hasMoreElements  boolean [CALL] java.util.Enumeration nextElement  java.lang.Object [CALL] java.util.Enumeration hasMoreElements  boolean [CALL] java.util.Enumeration nextElement  java.lang.Object [CALL] java.util.Enumeration hasMoreElements  boolean [CALL] java.util.Enumeration nextElement  java.lang.Object [CALL] java.util.Enumeration hasMoreElements  boolean [CALL] java.util.Enumeration nextElement  java.lang.Object [CALL] java.util.Enumeration hasMoreElements  boolean [CALL] java.util.Enumeration nextElement  java.lang.Object [CALL] java.util.Enumeration hasMoreElements  boolean [CALL] java.util.Enumeration nextElement  java.lang.Object [CALL] java.util.Enumeration hasMoreElements  boolean [CALL] java.util.Enumeration nextElement  java.lang.Object [CALL] java.util.Enumeration hasMoreElements  boolean [CALL] java.util.Enumeration nextElement  java.lang.Object [CALL] java.util.Enumeration hasMoreElements  boolean [CALL] java.util.Enumeration nextElement  java.lang.Object [CALL] java.util.Enumeration hasMoreElements  boolean [EXIT]").tokens

import numpy as np
from datasets import load_metric


acc_metric = load_metric("accuracy")
#wer_metric = load_metric("wer")
def compute_metric(eval_pred):
    #print(eval_pred)
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    #acc = metric.compute(predictions=eval_pred.predictions, references=eval_pred.label_ids)
    acc = acc_metric.compute(predictions=predictions, references=labels)
#     wer_metric = wer_metric.compute(predictions=predictions, references=labels)
    #print(acc)
    #print(acc_2)
    return {'accuracy': acc}#, 'perp': wer_metric}

In [78]:
from transformers import BertConfig, BertModel
bertmodel = BertForMaskedLM.from_pretrained('work/projsplit-train/test_trainer_bert_pre')

loading configuration file work/projsplit-train/test_trainer_bert_pre/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 768,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.15.0",
  "type_vocab_size": 5,
  "use_cache": true,
  "vocab_size": 10000
}

loading weights file work/projsplit-train/test_trainer_bert_pre/pytorch_model.bin
All model checkpoint weights were used when initializing BertForMaskedLM.

All the weights of BertForMaskedLM were initialized from the model checkpoint at work/projsplit-train/test_trainer_bert_pre.
If your task is similar 

In [80]:
bertmodel.config.output_hidden_states = True
assert bertmodel.config.output_hidden_states == True

In [65]:
from transformers import TrainingArguments
from transformers import DataCollatorForWholeWordMask
data_collator = DataCollatorForWholeWordMask(tokenizer=fast_tokenizer, mlm=True, mlm_probability=0.15)
training_args = TrainingArguments("test_trainer_bert_pre_3",
                                  overwrite_output_dir=True,
                                  run_name='monday_evening_1',
                                  report_to = 'wandb',
                                  #num_train_epochs=20,
#                                 prediction_loss_only=False,
                                  save_strategy='steps',
                                  save_steps=5000,
                                  save_total_limit=3,
#                                 eval_accumulation_steps=1000,
                                  logging_steps=1000,
                                  eval_steps=1000,
                                  max_steps=1000,
                                  evaluation_strategy='steps',
                                  per_device_train_batch_size=32,
                                  per_device_eval_batch_size=32,
                                  #per_device_eval_batch_size=4
)
from transformers import Trainer

trainer = Trainer(
    model=model,
    tokenizer=fast_tokenizer,
    data_collator=data_collator,
    args=training_args,
    train_dataset=encoded_dataset_train,
    eval_dataset=encoded_dataset_test,
    compute_metrics=compute_metric,
)

max_steps is given, it will override any value given in num_train_epochs


In [66]:
%env TOKENIZERS_PARALLELISM=true

env: TOKENIZERS_PARALLELISM=true


In [67]:
#%%wandb
trainer.train()
# train_result

#trainer.save_model()

The following columns in the training set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: text, func_name.
***** Running training *****
  Num examples = 3376066
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1000
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


/opt/conda/lib/python3.9/site-packages/transformers/data/data_collator.py:947: UserWarning: DataCollatorForWholeWordMask is only suitable for BertTokenizer-like tokenizers. Please refer to the documentation for more information.
  warnings.warn(


RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 23.65 GiB total capacity; 151.06 MiB already allocated; 2.44 MiB free; 170.00 MiB reserved in total by PyTorch)

In [93]:
ten_examples_google = load_dataset('text', data_files='work/projsplit/google.txt', split='train[:10000]').map(lambda x: {'func_name': x["text"].split(' ')[0]})

Using custom data configuration default-fd3e663e823696b8
Reusing dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-fd3e663e823696b8/0.0.0/d86c40dad297bdddf277b406c6a59f0250b5318c400bf23d420a31aff88c84c4)


  0%|          | 0/10000 [00:00<?, ?ex/s]

In [95]:
ten_k_google = ten_examples_google.filter(lambda x: x['func_name'] == 'compare')

  0%|          | 0/10 [00:00<?, ?ba/s]

In [101]:
len(ten_k_google.select(range(0,222)))

222

In [94]:
ten_examples_netty = load_dataset('text', data_files='work/projsplit/netty.txt', split='train[:10000]').map(lambda x: {'func_name': x["text"].split(' ')[0]})

Using custom data configuration default-0237c71beb1ce4db
Reusing dataset text (/home/jovyan/.cache/huggingface/datasets/text/default-0237c71beb1ce4db/0.0.0/d86c40dad297bdddf277b406c6a59f0250b5318c400bf23d420a31aff88c84c4)


  0%|          | 0/10000 [00:00<?, ?ex/s]

In [97]:
ten_k_netty = ten_examples_netty.filter(lambda x: x['func_name'] == 'compare')

  0%|          | 0/10 [00:00<?, ?ba/s]

In [98]:
len(ten_k_netty)

222

In [102]:

e = np.save("google_ex", bertmodel(**fast_tokenizer(ten_k_google.select(range(0,222))["text"],
                                   max_length = 512, truncation=True, padding=True, return_tensors="pt",
                                  )).hidden_states[-1].detach().numpy())
# print(e.shape)

    
    

In [89]:
bertmodel(**fast_tokenizer(ten_examples_google["text"],
                                   max_length = 512, truncation=True, padding=True, return_tensors="pt",
                                  )).hidden_states[-1].shape

torch.Size([10, 7, 512])

In [103]:
netty = np.save("netty_ex", bertmodel(**fast_tokenizer(ten_k_netty["text"],
                                   max_length = 512, truncation=True, padding=True, return_tensors="pt"
                                  )).hidden_states[-1].detach().numpy())

In [91]:
ds_with_embeddings = encoded_dataset_test.map(lambda example:
                            {'embeddings': np.array([0])})
                             #bertmodel(**fast_tokenizer(example["text"], max_length = 512, truncation=True, padding=True, return_tensors="pt")).hidden_states[-1][0].numpy()},
                                      #  batched=True)

# ds_with_embeddings = encoded_dataset_train.map(lambda example:
#                             {'embeddings': 
#                              bertmodel(**{'attention_mask': torch.tensor(example['attention_mask']),
#                                          'input_ids': torch.tensor(example['input_ids']),
#                                          'token_type_ids': torch.tensor(example['token_type_ids'])})[0].numpy()},
#                                         batched=True)

Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/text/default-fd3e663e823696b8/0.0.0/d86c40dad297bdddf277b406c6a59f0250b5318c400bf23d420a31aff88c84c4/cache-c7228d9b0bd8d370.arrow


In [73]:
torch.tensor(ds_with_embeddings[0]['embeddings']).shape

torch.Size([1])

In [75]:
ds_with_embeddings.add_faiss_index(column='embeddings')#, custom_index=index)
ds_with_embeddings.save_faiss_index('embeddings', 'embeddings_index_shard.faiss')

  0%|          | 0/2 [00:00<?, ?it/s]

In [97]:
index = faiss.IndexFlatL2(512)   # build the index
print(index.is_trained)

True


In [ ]:
index.search(hidden_states.detach().numpy(),k=1)

In [ ]:
from torch.nn import functional as F
import torch

In [ ]:
#subset = 
smaller_dataset = encoded_dataset_test.shard(num_shards=4000, index=0)

In [ ]:
text = "The capital of France, " + fast_tokenizer.mask_token + ",contains the Eiffel Tower."
input = fast_tokenizer.encode_plus(text, return_tensors = "pt").to("cuda")
mask_index = torch.where(input["input_ids"][0] == fast_tokenizer.mask_token_id)
logits = model(**input)
logits = logits.logits
softmax = F.softmax(logits, dim = -1)
mask_word = softmax[0, mask_index, :]
top_word = torch.argmax(mask_word, dim=1)
print(fast_tokenizer.decode(top_word))

softmax = F.softmax(logits, dim = -1)
mask_word = softmax[0, mask_index, :]
top_10 = torch.topk(mask_word, 10, dim = 1)[1][0]
for token in top_10:
   word = fast_tokenizer.decode([token])
   new_sentence = text.replace(fast_tokenizer.mask_token, word)
   print(new_sentence)

In [ ]:
smaller_dataset

In [ ]:
trainer_eval = trainer.evaluate(smaller_dataset, metric_key_prefix='no')

print(trainer_eval)
metrics = train_result.metrics
print(metrics)
#trainer.log_metrics("train", compute_metrics)
#trainer.save_metrics("train", compute_metrics)
#trainer.save_state()

print(metric.inputs_description)

In [ ]:
model_predictions = model(smaller_dataset)
final_score = metric.compute(predictions=model_predictions, references=gold_references)

In [ ]:
!ls  work/projsplit-train/

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="work/projsplit-train/test_trainer_bert_pre",
    tokenizer="work/projsplit-train/test_trainer_bert_pre"
)
result = fill_mask("setSuccessorInMultimap [ENTRY] void com.google.common.collect.LinkedHashMultimap$ValueEntry [MASK] [EXIT]")

print(result)

In [57]:
!mamba install --quiet --yes 'faiss-gpu'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Package           Version  Build                       Channel                    Size
──────────────────────────────────────────────────────────────────────────────────────────
  Install:
──────────────────────────────────────────────────────────────────────────────────────────

  + faiss             1.7.1  py39cuda102hb1acdd0_1_cuda  conda-forge/linux-64       2 MB
  + faiss-gpu         1.7.1  h788eb59_1                  conda-forge/linux-64      15 KB
  + libfaiss          1.7.1  cuda102hd37495c_1_cuda      conda-forge/linux-64      60 MB
  + libfaiss-avx2     1.7.1  cuda102h1234567_1_cuda      conda-forge/linux-64      60 MB

  Change:
──────────────────────────

In [59]:
import faiss

In [ ]:
faiss

In [ ]:
!mamba install --quiet --yes 'wandb'

In [189]:
netty.numpy()[0].shape
tsne_netty = TSNE(2, verbose=1)
tsne_proj_netty = tsne.fit_transform(netty.numpy()[0])

import pandas as pd
df = pd.DataFrame()
df["y"] = range(0,6)
df["comp-1"] = tsne_proj_netty[:,0]
df["comp-2"] = tsne_proj_netty[:,1]

sns.scatterplot(x="comp-1", y="comp-2", hue="y",
                palette=sns.color_palette("hls", 10),
                data=df).set(title="1 sentence")

/opt/conda/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


[t-SNE] Computing 5 nearest neighbors...
[t-SNE] Indexed 6 samples in 0.000s...
[t-SNE] Computed neighbors for 6 samples in 0.002s...
[t-SNE] Computed conditional probabilities for sample 6 / 6
[t-SNE] Mean sigma: 1125899906842624.000000
[t-SNE] KL divergence after 250 iterations with early exaggeration: 44.964821
[t-SNE] KL divergence after 450 iterations: 0.131007


In [171]:
import matplotlib.pyplot as plt 

In [169]:
tsne = TSNE(2, verbose=1)
tsne_proj = tsne.fit_transform(e.numpy()[0])
# Plot those points as a scatter plot and label them based on the pred labels

from matplotlib import cm
cmap = plt.cm.get_cmap('tab20')
fig, ax = plt.subplots(figsize=(8,8))
num_categories = 10
for lab in range(num_categories):
    indices = test_predictions==lab
    ax.scatter(tsne_proj[indices,0],tsne_proj[indices,1], c=np.array(cmap(lab)).reshape(1,4), label = lab ,alpha=0.5)
ax.legend(fontsize='large', markerscale=2)
plt.show()

/opt/conda/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


[t-SNE] Computing 6 nearest neighbors...
[t-SNE] Indexed 7 samples in 0.000s...
[t-SNE] Computed neighbors for 7 samples in 0.002s...
[t-SNE] Computed conditional probabilities for sample 7 / 7
[t-SNE] Mean sigma: 1125899906842624.000000
[t-SNE] KL divergence after 250 iterations with early exaggeration: 38.395630
[t-SNE] KL divergence after 1000 iterations: 0.158581


In [173]:
import seaborn as sns

In [176]:
df_subset = {}
df_subset['tsne-2d-one'] = tsne_proj[:,0]
df_subset['tsne-2d-two'] = tsne_proj[:,1]

In [181]:
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
#      hue="y",
    palette=sns.color_palette("hls", 10),
    data=df_subset,
    legend="full",
    alpha=0.3
)

ValueError: Could not interpret value `y` for parameter `hue`

In [190]:
test_input = fast_tokenizer(["performExpensiveLogSetup [ENTRY] void [CALL] java.io.PrintStream println java.lang.String void [EXIT] [CALL] java.lang.Math pow double|double double [CALL] java.lang.Math sqrt double double [CALL] java.io.PrintStream println java.lang.String void [EXIT]", "[EXIT]", ""], max_length = 512, truncation=True, padding=True, return_tensors="pt")
#fast_tokenizer.encode("performExpensiveLogSetup [ENTRY] void [CALL] java.io.PrintStream println java.lang.String void [EXIT] [CALL] java.lang.Math pow double|double double [CALL] java.lang.Math sqrt double double [CALL] java.io.PrintStream println java.lang.String void [EXIT]", max_length = 512, truncation=True, padding=True, return_tensors="pt")
hidden_states = bertmodel(**test_input).hidden_states


In [192]:
last_four_layers = [hidden_states[i] for i in (-1, -2, -3, -4)]
# cast layers to a tuple and concatenate over the last dimension
cat_hidden_states = torch.cat(tuple(last_four_layers), dim=-1)
print(cat_hidden_states.size())

# take the mean of the concatenated vector over the token dimension
cat_sentence_embedding = torch.mean(cat_hidden_states, dim=1).squeeze()
print(cat_sentence_embedding.size())

torch.Size([3, 10, 2048])
torch.Size([3, 2048])


In [32]:
bertmodel(**fast_tokenizer("performExpensiveLogSetup [ENTRY] void [CALL] java.io.PrintStream println java.lang.String void [EXIT] [CALL] java.lang.Math pow double|double double [CALL] java.lang.Math sqrt double double [CALL] java.io.PrintStream println java.lang.String void [EXIT]", max_length = 512, truncation=True, padding=True, return_tensors="pt"))[1][0].numpy().shape

(1, 10, 512)

In [191]:
test_input

{'input_ids': tensor([[1, 0, 6, 0, 5, 0, 0, 0, 5, 0],
        [1, 5, 0, 3, 3, 3, 3, 3, 3, 3],
        [1, 0, 3, 3, 3, 3, 3, 3, 3, 3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}

In [62]:
inputs = {
    "input_ids": encoded_dataset_test[0],
   #"attention_mask": batch[1]
}
input_sentence = torch.tensor(fast_tokenizer.encode("performExpensiveLogSetup [ENTRY] void [CALL] java.io.PrintStream println java.lang.String void [EXIT] [CALL] java.lang.Math pow double|double double [CALL] java.lang.Math sqrt double double [CALL] java.io.PrintStream println java.lang.String void [EXIT]", max_length = 512, truncation=True, padding=True)).unsqueeze(0)
output = bertmodel(input_sentence)
logits = output[0]
hidden_states = output[1]

In [67]:
input_sentence.shape

torch.Size([1, 10])

In [69]:
logits.shape

torch.Size([1, 10, 10000])

In [54]:
output.hidden_states[0]

tensor([[[-9.5000e-01,  1.1246e+00,  6.5500e-01,  ...,  9.4959e-01,
           9.6691e-01, -5.9020e-01],
         [ 2.1861e-01,  7.6641e-01, -9.3439e-01,  ...,  1.2487e-01,
          -2.1340e+00, -1.6859e+00],
         [ 1.1559e-01,  4.6041e-01,  9.1597e-01,  ..., -9.9334e-01,
          -1.4955e+00, -4.6213e-01],
         ...,
         [-1.8021e-01,  9.4698e-02,  2.3828e-01,  ..., -2.2032e-02,
          -7.0602e-01, -1.0442e+00],
         [ 2.5412e+00, -7.9635e-01, -5.2533e-01,  ..., -2.5971e-01,
          -1.4559e+00,  3.3725e-01],
         [-1.2266e+00, -1.3498e+00, -2.6052e-01,  ..., -1.0849e-01,
           1.5142e-03, -5.4500e-01]]])

In [63]:
len(hidden_states)

7

In [23]:
hidden_states[0].shape

torch.Size([1, 10, 512])

In [24]:
hidden_states[1].shape

torch.Size([1, 10, 512])

In [46]:
hidden_states[6]

tensor([[[-1.9685,  1.0561,  0.7994,  ...,  0.1802, -0.0625, -1.7218],
         [-0.1738,  1.0194, -0.7157,  ...,  1.0432, -2.3895, -2.2104],
         [-0.6670,  1.8940,  1.0295,  ..., -0.5803, -2.1574, -0.5008],
         ...,
         [-0.6804,  0.4010,  0.9084,  ...,  0.7233, -1.1843, -1.7977],
         [ 1.6336, -0.3833, -0.3888,  ..., -0.0906, -2.3090, -0.2373],
         [-1.1869, -0.4380,  0.3823,  ...,  0.4377, -1.4365, -1.5924]]])

In [24]:
logits.shape

torch.Size([1, 10, 10000])

In [25]:
hidden_states.shape

AttributeError: 'tuple' object has no attribute 'shape'

In [102]:
import importlib.util

In [103]:
importlib.util.find_spec("faiss")

ModuleSpec(name='faiss', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7f45201b6730>, origin='/opt/conda/lib/python3.9/site-packages/faiss/__init__.py', submodule_search_locations=['/opt/conda/lib/python3.9/site-packages/faiss'])

In [79]:
smaller_dataset = encoded_dataset_train.shard(num_shards=2000, index=0)

In [234]:
smaller_dataset

torch.set_grad_enabled(True)